# Introduction to Julia


# Agenda

1. **What is Julia?**
2. **What can you do with the Julia programming language?**
    - Scientific Modelling
    - Fast Linear Algebra
3. Basics
    - Tools and Getting Around
    - Literals, Variables, and Functions
    - Program Control Flow
    - Collections
    - Modules and Packages: How to Use Other People's Code
4. Project: Writing a Root Finding Program
    - Overview of Newton's Method
    - Handling Derivatives
    - Adding Visual Feedback
5. Additional Topics
    - Broadcasting
    - More on Types, Immutability, Type Stability
    - More on Function Arguments and Methods
    - Multiple Dispatch

# Learning Goals

After this workshop you should:

1. Be familiar with Julia syntax.
2. Understand variables, functions, and basic data types in Julia.
3. Be able to write short programs in Julia.
4. Feel confident enough to explore more on your own.

# What is Julia?

> **The Julia Project as a whole is about bringing usable, scalable technical computing to a greater audience**: allowing scientists and researchers to use computation more rapidly and effectively; letting businesses do harder and more interesting analyses more easily and cheaply.
> Part of that project entails creating a free, open-source language that is as easy to use as possible, so that researchers who are not necessarily professional programmers can easily create and share programs that others will be able to use for free and improve upon.
> However, a large part of the project is also about creating an ecosystem in which such openness and sharing can take place.

-- [Julia Project page](https://julialang.org/project/)

### Why should I care?

1. It is easy to prototype *fast*, *idiomatic* code in Julia *at a high level* thanks to Julia's JIT compiler and powerful type system.
2. Julia favors a generic programming style: write functions that assume as little as possible and compose them to write powerful programs.
3. The Julia community is open and highly collaborative. Many high profile packages represent the joint effort of several brilliant people.

### How is it different from `xxxxx`?

Syntax comparison between MATLAB, Julia, and Python: https://cheatsheets.quantecon.org/
The Julia manual has a dedicated package on this: https://docs.julialang.org/en/v1/manual/noteworthy-differences/

### I like `X/Y/Z` from `xxxxx` language. I don't think I can ever switch.

Early on in the development of Julia, people put effort into building interoperability between Julia and other languages.
This means that if you really need a package implemented in another language, it is possible to use it directly from within Julia.

**Examples**

- Python: Use [PyCall.jl](https://github.com/JuliaPy/PyCall.jl). 
- R: Use [RCall.jl](https://github.com/JuliaInterop/RCall.jl).
- MATLAB: Use [MATLAB.jl](https://github.com/JuliaInterop/MATLAB.jl).
- FORTRAN or C: Use the built-in `ccall` function documented [here](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/).
- C++: Use [Cxx.jl](https://github.com/JuliaInterop/Cxx.jl).

The [JuliaInterop](https://github.com/JuliaInterop) organization has more packages available for other languages.

**Ultimately, the Julia language is (a very useful) tool. You should use whatever makes you productive, and your work reproducible.**

### How does it work?

Credit: Arch D. Robison

|   |   |
|---|---|
| <img src="julia_toolchain.png" alt="julia_toolchain.png" width="400"/> | <img src="julia_introspect.png" alt="julia_introspect.png" width="530"/> |

**Big Picture**: Julia's *design* uses a robust type system to pass type information to a compiler that can then generate efficient machine code.

The type system uses *multiple dispatch* to help programmers write generic type-stable code; type-stable code is easy for the compiler to reason about.

On the surface, Julia syntax looks like any other scripting language because it is quite human readable.
However, Julia code is much more explicit than it appears on the surface.

Dr. Chris Rackauckas has [an excellent page](https://ucidatascienceinitiative.github.io/IntroToJulia/Html/JuliaMentalModel) on developing a mental model for Julia:

> **A Mental Model for Julia: Talking to a Scientist**
> * When you're talking, everything looks general. However, you really mean very specific details determined by context.
> * You can quickly dig deep into a subject, assuming many rules, theories, and terminology.
> * Nothing is hidden: if you ever want to hear about every little detail, you can ask.
> * They will get mad (and throw errors at you) if you begin to be loose with the specific details.

Let's look at an example.
The following function computes the sum
$$x_{1} + x{2} + \ldots + x_{m} + y_{1} + y_{2} + \ldots + y_{n}$$

In [ ]:
function mysum(x, y)
    s = zero(x[1])

    for i in eachindex(x)
        s = s + x[i] # sum up elements of x
    end

    for j in eachindex(y)
        s += y[j]    # sum up elements of y
    end

    return s
end

First check that the function works as intended.

In [ ]:
x = rand(5) # random vector with 5 elements [x_1, x_2, ..., x_5]
y = rand(8)
mysum(x, y) == sum(x) + sum(y)

It works when the inputs are different collection types; i.e. `Vector` vs `Tuple`.

In [ ]:
x = [2, 4, 6] # vector
y = (1.0, 3.0, 5.0) # tuple
mysum(x, y) == sum(x) + sum(y)

It works for scalars.

In [ ]:
mysum(1, 1) == 1 + 1

It works for vectors of vectors

In [ ]:
x = [rand(3), rand(3)]
y = [rand(3)]
mysum(x, y) == x[1] + x[2] + y[1]

But it fails here and Julia tells you why

In [ ]:
x = [rand(3), rand(3)] # Vector of vectors 
y = [(1.0, 3.0, 5.0)] # Vector of tuples
mysum(x, y) == x[1] + x[2] + y[1]

What does Julia do differently in each case?
You can ask with tools like `@code_warntype` and `@code_llvm` to see for yourself.

In [ ]:
@code_warntype mysum(1, 1)

In [ ]:
@code_llvm mysum(1, 1)

In [ ]:
@code_warntype mysum(rand(3), rand(4))

In [ ]:
@code_llvm mysum(rand(3), rand(4))

**Takeaway**: You can be verbose or terse with the Julia language, but it's best to be concise.

# What can I do with the Julia programming language?

### Scientific Modeling

**Credit**: [epirecipes](https://github.com/epirecipes/sir-julia/blob/master/markdown/ode/ode.md)

OrdinaryDiffEq.jl is part of the SciML ecosystem.
See the [organization's website](https://sciml.ai/) for more details.
DifferentialEquations.jl is the main package with lots of extra features at the cutting edge of scientific machine learning.

The following code implements a [Susceptible-Infected-Removed (SIR) model](https://github.com/epirecipes/sir-julia) for infectious diseases.
- Infected individuals spread disease through 'contact' with susceptibles.
- The rate at which individuals are infected is proportional to the proportion of infecteds in a population; it depends on an infection risk per contact, $\beta$, and an average contact rate, $c$.
- Infecteds recover at a rate $\gamma > 0$, with $1/\gamma$ denoting an average recovery time.

In [ ]:
using OrdinaryDiffEq, Plots

function sir_ode!(du, u, p, t)
    (S, I, R) = u
    (β, c, γ) = p
    N = S+I+R
    @inbounds begin
        du[1] = -β*c*I/N*S
        du[2] = β*c*I/N*S - γ*I
        du[3] = γ*I
    end
    nothing
end

# Initial conditions
u0 = [990.0,10.0,0.0]; # S,I.R

# Model parameters
p = [0.05,10.0,0.25]; # β,c,γ

# Timespan
δt = 0.1
tmax = 40.0
tspan = (0.0,tmax)
t = 0.0:δt:tmax

# Create ODE problem and solve it
prob_ode = ODEProblem(sir_ode!, u0, tspan, p)
sol = solve(prob_ode, Tsit5())

# Plot the solution
plot(sol, xlabel="time, t", ylabel="population, u(t)", label=["S" "I" "R"], legend=:outerright)

### Fast Linear Algebra

[BLAS](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms) and [LAPACK](https://en.wikipedia.org/wiki/LAPACK) are important libraries for carrying out *fast and correct* linear algebra operations on computers.
There are various implementations written in 'low-level' languages like C or Fortran, sometimes tuned for specific hardware, dating back to 1979.

Developers in Julia are exploring the possibility of implementing linear algebra subroutines within the Julia language.
The following compares the default BLAS implementation (OpenBLAS) shipped with Julia against [Octavian.jl](https://github.com/JuliaLinearAlgebra/Octavian.jl).
A more detailed look (with different hardware) can be found [here](https://github.com/JuliaLinearAlgebra/Octavian.jl/issues/24#issuecomment-763704038).

Run a separate Julia process using the `linalg.jl` script.
It takes in two arguments, # BLAS threads and matrix size.

In [ ]:
run(`/home/alanderos/julia-1.6.1/bin/julia --project=@. linalg.jl 10 1000`)

Run a separate Julia process using the `octavian.jl` script.
It requires you specify the number of threads (`-t 10`; match to # physical cores) and matrix size (`1000`).

In [ ]:
run(`/home/alanderos/julia-1.6.1/bin/julia --project=@. -t 10 octavian.jl 1000`)

In [ ]:
versioninfo(verbose=true)

**Note**: OpenBLAS may not always be able to detect the correct architecture until it is updated, and so it will default to the next best available subroutines.
[The codename Prescott dates back to Intel's Pentium 4 from the 00s](https://ark.intel.com/content/www/us/en/ark/products/codename/1791/prescott.html).
In reality the gap may be closer than reported here.

**Note**: Many programming languages for statistics, data science, or other technical computing use a BLAS implementation like OpenBLAS (R for example).

In [ ]:
using LinearAlgebra
BLAS.openblas_get_config()

# Basics

### Tools to write Julia code

Julia program files end with the `.jl` file extension.
There are two environments people generally use to write Julia code:

1. An *interactive* environment lets you input commands and gives you feedback as you work. For example, Julia comes with its own **REPL (Read-Evaluation-Print Loop)** that operates in a command line fashion. 
2. A *non-interactive* environment executes a series of commands (program) in the background without any feedback. Error messages, due to logical errors or program state, are generally handled by the program itself.

**Programs** or **scripts** are usually, but not always, non-interactive in that they are used to automate long tasks without active control from a user.

Here we will focus on how to work within a Jupyter notebook.
Much of what we do here will carry over to working in a REPL.

### Jupyter notebooks

This notebook is made up of different cells:

1. **Code cells** contain a series of instructions (in Julia). You can run them by clicking the run button, entering `Ctrl + Enter`.
2. **Markdown cells** contain text that can be formatted with [Markdown](https://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html). They may contain text, images, LaTeX equations.

After selecting a cell (highlighted in green):

* You can insert a new cell (code cell by default) by typing `Esc` followed by `A` (insert above) or `Esc` followed by `B` (insert below).
* Convert it to a code cell using `Esc` then `Y`.
* Convert it to a Markdown cell using `Esc` then `M`.

**Exercise**: Convert the following cell into a code cell and run it.

sqrt(2)

Another tip: You can toggle code lines with `Esc`, `L`.

### Getting Around

So far we don't really know how to do anything in Julia. Thankfully, there is a built-in help system. You can access it by entering `?`:

In [ ]:
?

Note that there are four different "modes" available in both a REPL and Julia notebook:

- Default: Julia command mode.
- `?` (`help?>`): Access the help system.
- `;` (`shell>`): Access a command line shell for your system.
- `]` (`xxx pkg>`): Access Julia's package system.

You can return to the default by hitting `Delete` or `Backspace`.
A Julia REPL will tell you which mode you are in as you type a command.

You can us the help system to ask about any object defined within your working environment including data types, functions, and variables.

In [ ]:
?print

The `apropos` function is also useful for finding existing commands by searching documentation for keywords.

In [ ]:
apropos("sorting")

### Literals, Variables, and Basic Data Types

In most programming languages, `1`, `1.0`, and `"1"` are likely to be different kinds of objects even though they appear similar to each other.

In [ ]:
1

In [ ]:
1.0

In [ ]:
"1"

Such objects are said to *belong to a class* or to *be of a certain type*. 
**In Julia, we think of objects in terms of types**. 

There is built-in support for various number types that are important for computing:

In [ ]:
typeof(0x01) # 8-bit unsigned integer

In [ ]:
typeof(1) # 64-bit integers, or 32-bit if you're using an older computer

In [ ]:
typeof(1.0) # double precision floating point

In [ ]:
typeof(1f0) # single precision floating point

In [ ]:
typeof('1') # characters, only one symbol

In [ ]:
typeof("1") # strings, sequence of characters

In [ ]:
typeof(true) # boolean: true or false

Symbols like `1` and `3.14` are often referred to as *literals* because the notation itself literally bears its meaning within the language.
In contrast to literals, we can use certain symbols to define *variables*.
The general syntax is `<variable name> = <assigned value>`:

In [ ]:
myvariable = 2.0
myothervar = 3.4; # the semicolon, ;, is used to suppress printing the result of this code cell

In [ ]:
myvariable

In [ ]:
myothervar

As with many programming languages, the symbol `=` is reserved for variable assignment.
Variables are useful because we can use them to store values to later use in an algorithm, or a sequence of steps.
Like the literals we looked at before, variables also have types depending on the object it is *bound to*:

In [ ]:
typeof(myvariable)

One nice feature in Julia is its [support for Unicode characters](https://docs.julialang.org/en/v1/manual/unicode-input/) in function names and variables.
In a REPL or Jupyter notebook (using Julia), you can input special characters using tab completions.
Here are a few examples:

In [ ]:
α = 0.5; # type \alpha → Tab
x₁ = 42; # type x\_1 → Tab
🍔 = ["bun", "patty", "bun"]; # type \:hamburger: → Tab

Try printing the values of these variables below:

### Everything in Julia has a return type

This includes functions, `begin ... end` blocks, `for` loops, expressions, and `if .. else` statements.
If there is no "reasonable" value to return, Julia will return `nothing`, an alias for the `Nothing` object.

In [ ]:
nothing

In [ ]:
typeof(nothing)

In [ ]:
?nothing

In [ ]:
c = 0.0

val = for i in 1:3 # for loops evaluate to nothing
    c += rand()
end

@show val
@show c

In [ ]:
val = if c < 1 # if statements return the last value in the branch taken
    "abc"
    "def"
else
    "ghi"
    "jkl"
end

val

Keep this in mind as we move on to functions and control flow topics.

### Functions

So far, we have *literals* to represent basic units of computation and *variables* to store values within a program.
These are the nouns in a programming language.
How do we actually say anything useful?
We need *functions*, which serve as verbs.

#### The usual [artihmetic operators](https://docs.julialang.org/en/v1/manual/mathematical-operations/#Arithmetic-Operators) (which are functions) are supported in Julia:

In [ ]:
1 + 1 # addition

In [ ]:
2.3 * 4 # multiplication

In [ ]:
9 / 3 # (floating point) division

In [ ]:
7 % 2 # remainder after integer division

In [ ]:
2 ^ 4 # exponentiation

In [ ]:
-1 # negative sign

#### Other [logical operators](https://docs.julialang.org/en/v1/manual/mathematical-operations/#Boolean-Operators) and [comparison operators](https://docs.julialang.org/en/v1/manual/mathematical-operations/#Numeric-Comparisons):

In [ ]:
1 < 2 & 2 < 2 # parantheses not needed

In [ ]:
!(1 ≤ 1) # type `\le` and hit `TAB` to get the ≤ symbol

#### Printing (note that the character `\n` indicates a newline):

In [ ]:
print("The value of myvariable is ", myvariable, ".\n")
print("The value of myothervar is ", myothervar, ".\n")

The same can be achieved with `println`:

In [ ]:
println("The value of myvariable is ", myvariable, ".")
println("The value of myothervar is ", myothervar, ".")

Multi-line strings, `"""..."""`, interpret whitespace.
The `$` symbol can be used to interpolate a variable's value into a string.
More generally, `$(ex)` will interpolate the value of an expression `ex`.

In [ ]:
print(
"""
The value of myvariable is $myvariable.
1 + 1 is $(1 + 1).
"""
)

**Note:** The terms *operator* and *function* are virtually the same for our purposes.
The only meaningful distinction from our POV is that *operators* have shorthand names based on punctuation marks whereas *functions* are typically spelled out in several characters.
Binary operators also support *infix notation* so we can use them just how one typically writes down math but it's also possible to call operators the same way one invokes `println`:

In [ ]:
+(1, 1)

Notice that `1 + 1` returns `2` as an `Int64` whereas as `4 / 2` returns `2` as a `Float64`.
We can try a few more examples to see that *addition on two elements of the same type* always returns an answer of that same type (in fact, this is a mathematical property of addition on certain sets).

What happens when we mix data types?

Julia has a set of promotion rules to determine how to resolve certain operations.
For example, `1 + 3.2` will return `4.2` as a `Float64`.
Test this yourself using `typeof`.

In [ ]:
# your code here

This is not the case with division and so we naturally have a few ways to handle it: `/`, `div`, and `//`.
Check the documentation for these functions.

In [ ]:
# your code here

Julia's type promotion rules make it is so programmers do not waste time casting objects to 'compatible' data types; they are already compatible.
Importantly, operations with numbers already handle type promotion with reasonable defaults so that standard arithmetic "just works" out of the box.

However, if you ever needed to handle conversions explicitly you can do it with the `convert` function:

In [ ]:
convert(Complex{Float64}, 1.0) # specify the type, then give the object to convert

Many useful functions are already implemented in Julia.
In fact, if there is something you want to do it is often helpful to try looking up `?<verb>` to see if an existing function can solve your task.

Programming often involves building up many small programs that, ideally, can be composed to write larger more complex programs.
There are many ways to define a function in Julia.
For example, consider the function $x \mapsto x + 1$ that takes a number and adds 1 to it:

In [ ]:
f(x) = x+1    # written as a single line

function f(x) # written as multiple lines
    x+1
end

**Aside** Notice that a few words in the definition are highlighted.
The words `function` and `end` are highlighted *purple* because they are **reserved words** -- notation that the Julia programming language reserves for the language itself.
We can understand the syntax as follows:
- `function`: begin a statement (possibly multiple lines) that defines a function.
- `end`: mark the end of the last statement.
There are many more reserved words, listed here in the [Julia documentation](https://docs.julialang.org/en/v1/base/base/#Keywords).

Getting back to function definitions, the general syntax is

In [ ]:
function FUNCTION_NAME(INPUTS_SEPARATED_BY_COMMAS)
    # function body
end

By default, a function returns the value of the last statement within the body.
You can be specific about what the return value should be by using the `return` keyword.

**Exercise**: What is the return value of the following functions?

In [ ]:
function fun1(x)
    x + 1
    x + 2
    x + 3
end

In [ ]:
function fun2(x)
    x + 1
    return x + 2
    x + 3
end

In [ ]:
function fun3(x)
    x + 1
    x = x + 2
    return x + 3
    x = x + 4
end

In [ ]:
function fun4(x)
    x + 10
    return
    x - 2
end

#### Exercises

Write a function that evaluates the quadratic $a x^{2} + b x + c$.
Your function should take four (4) inputs, `a`, `b`, `c`, and `x`.

In [ ]:
function eval_quadratic()
end

A few test cases:

In [ ]:
@show eval_quadratic(0, 0, 2.0, 100.0)   # == 2
@show eval_quadratic(1.0, 0, 0, 10.0)    # == 100
@show eval_quadratic(0, 1.0, 0, 10.0)    # == 10
@show eval_quadratic(2, 1, -10, 2)       # 0

Write a function that computes both roots of a quadratic,
$$
a x^{2} + b x + c = 0 \iff x = \frac{-b \pm \sqrt{b^{2} - 4 a c}}{2a}
$$
Your function should take three (3) inputs, `a`, `b`, and `c`, which you may assume are numbers of the same type.
You can also assume $|a| > 0$.
Return the roots as a tuple; e.g. `return root1, root2` or `return (root1, root2)`.

In [ ]:
function quadratic()
end

A few test cases:

In [ ]:
@show quadratic(1.0, -2.0, 1.0) # == (1.0, 1.0); x^2 - 2x + 1
@show quadratic(1.0, 0, -1.0)   # == (-1.0, 1.0); x^2 - 1

**A note for scientists**:
Try the following code using your `quadratic` function

In [ ]:
a = 1.0; b = 100.0; c = 1.0
r1, r2 = quadratic(a, b, c)
@show eval_quadratic(a, b, c, r1)
@show eval_quadratic(a, b, c, r2)

Depending on how you chose to implement your function, one of the results above is not $0$ but close to it.
In this case, $b^{2}$ is much larger compared to $4ac$ which means $b^{2} - 4ac$ is subject to *roundoff error*.
This happens because numbers on the real line *cannot* be represented exactly on a computer.
It is an issue for irrational numbers like $\sqrt{2}$ and even rational numbers like $1/10 = 0.1$:

In [ ]:
bitstring(0.1) # this is not exact in base 2!

Julia uses the [IEEE 754-2008 standard](https://en.wikipedia.org/wiki/IEEE_754-2008) to handle floating-point arithmetic.
The Julia manual has plenty of [references](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Background-and-References) for the interested reader. 

Some useful values and functions to know about:

- `Inf`/`Inf32`/`Inf16`: Literals for representing infinity.
- `0.0` and `-0.0`: Signed zeros. note that `0.0 == -0.0`.
- `NaN`: Not a Number; used to handle numerical issues like division by `0` or loss of precison due to underflow/overflow.
- `exponent`: Return the exponent in the floating point representation of a number.
- `significand`: Return the digits in the floating point representation of a number.
- `signbit`: Check the sign bit of a number (false for positive, true for negative).
- `bitstring`: Return the binary representation of a number literal.
- `eps`: Machine epsilon; the gap between `1.0` and the next largest representable value of the same numeric type.
- `nextfloat`/`prevfloat`: The next/previous representable floating-point number relative to an input.

**Takeaway**: Try to look through documentation to see if a function you want has already been implemented, *especially* for any numerics.

### Program Control Flow

*Conditional statements* and *repeated evaluation* are common patterns that emerge in algorithms.
Julia handles these concepts in a straightforward way.

#### Conditional Statements

`if-else` blocks allow you to execute a block of code conditional on some requirement.

In [ ]:
if rand() < 0.5 # mimic a coin flip
    println("Heads")
else
    println("Tails")
end

These statements can be changed using `if-elseif-else` blocks.
Remember that code blocks always have a return value.

In [ ]:
u = rand()

x = if u < 0.25
    1
elseif 0.25 ≤ u < 0.5
    2
elseif 0.5 ≤ u < 0.75
    3
else
    4
end

println("The value of x is $(x).")

You can omit the `elseif` or `else` portions altogether.
These blocks can be written as one-liners, too.

What is the return type when we do assignment like this?
Use `@show x` to investigate.

In [ ]:
x = if rand() < 0.5 "In the if statement" end

@show x

The ternary operator (3-argument) `?` can be used to write `if-else` statements more compactly as

`<condition> ? <true branch> : <false branch>`

In [ ]:
truth_value = rand() < 0.5

truth_value ? println("branch taken if true") : println("branch taken if false")

@show truth_value

Lastly, the [short-circuit operators](https://docs.julialang.org/en/v1/manual/control-flow/#Short-Circuit-Evaluation), `&&` and `||`, can also be used to write compact conditional statements.
Statements chained with these operators are evaluated left to right.

* Short-circuit AND (`&&`): In `a && b`, `b` is evaluted only when `a` returns `true`.
* Short-circuit OR (`||`): In `a || b`, `b` is evaluated only when `a` returns `false`.

For example, in the Julia documentation there is an example that uses them to implement a recursive version of a factorial function.

In [ ]:
function fact(n::Int) # ::Int is a type annotation; we'll discuss this later
    n >= 0 || error("n must be non-negative") # use || to detect a violated condtion
    n == 0 && return 1 # use && to detect the base case
    n * fact(n-1) # otherwise, continue to recurse
end

In [ ]:
fact(0)

In [ ]:
fact(-1)

In [ ]:
fact(4)

#### Repeated evaluation with loops

`for` loops in Julia allow you to repeat a set of instructions a specific number of times.
For example, the following code prints out a sequence of numbers:

In [ ]:
for i = 1:10 # this is MATLAB/FORTRAN/C/C++ -like style
    println("The value of i is $(i)")
end

We start by giving some condition that determines how many times to execute commands, in this case we define a *local* variable `i` and state that it will take values from `1` to `10` using the range object `1:10` (will discuss later).
Next, we give a command to execute repeatedly.
The `for` loop will run until it exhausts values in the "enumeration".

Another way is to pass a list and iterate over the elements of that list.

In [ ]:
mylist = collect(1:10) # creates a 1-D array with elements 1 through 10; discuss later!

for item in mylist # or for i in 1:10; this is the preferred style in Julia
    println("The value of item is $(item)")
end

You can use `enumerate` to grab both the index and member at the same time:

In [ ]:
mylist = rand('a':'z', 10)

for (i, item) in enumerate(mylist)
    println("The item at index $(i) is $(item)")
end

For loops will even work with dictionaries (discussed later!):

In [ ]:
d = Dict('a' => 1, 'b' => 2, 'c' => 100)

In [ ]:
for (key, value) in d
    println("The key $(key) maps to $(value)")
end

#### Exercise

Implement the factorial function using a `for` loop.

In [ ]:
function fact2(n::Int)

end

Compare against the recursive version `fact` defined earlier, and the Julia standard `factorial`.

In [ ]:
fact(21)

In [ ]:
fact2(21)

In [ ]:
factorial(21)

The `while` loops allows one to execute repeated evaluation without knowing how many times to run a set of instructions ahead of time.
Instead of iterating over a set of values, a `while` loop runs until some condition evaluates to `false`:

In [ ]:
counter = 0

while counter < 5
    println("The current value is $(counter)")
    counter = counter + 1 # or counter += 1
end

In [ ]:
counter = 0
still_true = true # use a Bool as a flag to indicate current state

while still_true
    println("The current value is $(counter)")
    counter = counter + 1    # make sure to update the counter
    still_true = counter < 5 # make sure to update the variable we use to check
end

Care must be taken to make sure a `while` loop eventually terminates.
If you accidentally make an infinite `while` loop, just use the "Interrupt the kernel" button (□), `Esc` → `C`, or `Ctrl + C`.

#### Exercises

Credit: [Algorithms from THE BOOK](https://locus.siam.org/doi/book/10.1137/1.9781611976175?mobileUi=0&) by Kenneth Lange

The ancients could compute square roots long before the age of computers or the rigorous formulation of the real numbers.
The so-called Babylonian method estimates square roots via the iterative method
$$
x_{k+1} = \frac{1}{2}\left[x_{k} + \frac{a}{x_{k}}\right].
$$

Implement a function that computes the square root of an a non-negative number `a` based on an initial guess `x0`.
Your function should run until it achieves a given relative tolerance, $\epsilon$ = `tol`; in symbols this is
$$
|x_{k+1} - x_{k}| < \epsilon~(x_{k} + 1).
$$
Your function should also report the number of iterations taken, and return the estimate.

In [ ]:
function babylonian(a, x0, tol)
    x_cur = x0            # current estimate
    x_old = x0            # previous estimate
    not_converged = true  # flag that indicates convergence
    iter = 0              # number of iterations
    
    # your code goes here
    while not_converged
        x_old = x_cur
        x_cur = 0.5 * (x_cur + a / x_cur)
        not_converged = abs(x_cur - x_old) ≥ tol * (x_old + 1)
        iter += 1
    end
    
    println("Converged after $(iter) iterations.")
    
    return x_cur
end

In [ ]:
x_babylonian = babylonian(2, 1.4, 1e-6)

In [ ]:
x_sqrt = sqrt(2)

In [ ]:
abs(x_babylonian - x_sqrt)

Try computing the square root of `0.0` or `0` using your implementation.

Write a function that counts the number of times a character `char` occurs in a given string `str`.
It should treat the upper and lower case characters as being the same; e.g. 'c' and 'C' count as the same letter.
**Hint**: Julia allows aribtrary objects to implement some notion of *iteration* and *indexing*.
A `String` object is both iterable (so that `for c in str` makes sense) and indexable (so that `str[1]`).

In [ ]:
function countin()

end

You can use the following function to help you test your implementation.
It converts "this" into "tHis".

In [ ]:
function mockify(str)
    output = ""
    for (i, c) in enumerate(str)
        if i % 2 == 0
            output *= uppercase(c)
        else
            output *= lowercase(c)
        end
    end

    return output
end

In [ ]:
test_string = "The quick brown fox jumps over the lazy dog."
countin('a', test_string) # should equal 1

In [ ]:
countin('o', mockify(test_string)) # should equal 4

### Collections

Roughly speaking, a *collection* is simply an object that stores multiple objects; e.g. an object that stores multiple literals.
There are several important collections to know in Julia.
Note that collects can be nested within each other, so it is possible to have an `Array` of `Array`s or a `Dict` of `NamedTuple` objects.

Before diving into specific examples, you should be aware that collections share so much behavior that Julia defines [interfaces](https://docs.julialang.org/en/v1/manual/interfaces/) to make it easy to work with types that implement these common behaviors.
Examples of interfaces include:

* *indexing* to access a particular element in a collection, and
* *iteration* to retrieve data from a collection in a predictable fashion (e.g. first, second, third...).

#### Array

A Julia [`Array`](https://docs.julialang.org/en/v1/base/arrays/) is an indexed collection with a particular shape determined by its dimensions.
This means the elements of an `Array` are ordered; it makes sense to speak of "the first element" or "the last element". Here are a few examples:

In [ ]:
rand(3)

In [ ]:
rand(3, 3)

In [ ]:
rand(3, 3, 3)

The `Array` type is actually a parametric type -- it defines a *family of types* parameterized by the element type and number of dimensions.
Specifically, we have `Array{ElementType, NumberDimensions}`:

In [ ]:
?Array

Note that `Array{T,1}` is an alias for `Vector{T}` and `Array{T,2}` is an alias for `Matrix{T}`.

Instead of generating random arrays, we can create arrays with specific contents using square brackets `[]`:

In [ ]:
x = [10.0, 3.2, sqrt(2)]

You can access elements of an `Array` using `x[<index>]`.
**Note that Julia arrays start at index `1`!!!**.

If you really hate this for some reason, or your algorithm needs special indexing, you can write your own array type just like in [OffsetArrays.jl](https://github.com/JuliaArrays/OffsetArrays.jl).

In [ ]:
x[1]

You can also create matrices by separating elements in a row by spaces, and indicate a new row by a `;`.

In [ ]:
y = [1 2 3; 4 5 6; 7 8 9]

You can specify a common type, too:

In [ ]:
Float32[1 2 3; 4 5 6; 7 8 9]

You can use a linear index, e.g. `y[2]`, or coordinates, e.g. `y[2,1]` to acces the element `4`:

In [ ]:
y[2] == y[2,1]

**Julia arrays are column-major, meaning elements are organized as contiguous data along columns**.
This means that if you ever need to iterate over the elements of a multi-dimensional arrays, the "fast" dimension should be the first dimension.

In [ ]:
for j in 1:3, i in 1:3 # (j,i) order is favored over (i,j) order
    println("y[$i,$j] = $(y[i,j])")
end

Lastly, you can select all the elements along a particular dimension using a colon `:`:

In [ ]:
y[:, 1] # selects column 1

In [ ]:
y[2, :] # selects row 2

Julia arrays are mutable, so you can change the contents of a particular location in the array using the assignment operator `=`:

In [ ]:
y[3,3] = 100

In [ ]:
y

Other operations that can mutate an array:

In [ ]:
x = Int64[] # empty array of Int64 objects

In [ ]:
for j in 1:5
    push!(x, j) # add an element at the end
end

x

In [ ]:
pop!(x); x # delete the last element

In [ ]:
y = rand(3)
[x; y] # vertical concatenation; equivalent to vcat(x, y)

In [ ]:
y = rand(length(x))
[x y] # horizontal concatenation; equivalent to hcat(x, y)

An `Array` has certain properties, like `length`, `size`, and `eltype`:

In [ ]:
A = rand(10, 12)

In [ ]:
length(A)

In [ ]:
size(A)

In [ ]:
eltype(A)

#### Ranges

A range is an object that behaves much like an `Array` in that it supports indexing and iteration.

#### Tuples

Tuples are similar to arrays in that they are lists of objects, but they do not have a shape.
They are creatd using parantheses, `()`.

In [ ]:
x = (1, 2, 3)

Accessing elements works the same as an `Array`:

In [ ]:
x[2]

But a `Tuple` is immutable

In [ ]:
x[2] = 100

So why would you use `Tuple`?
Because a `Tuple` is immutable, it can be allocated on the stack -- its size is known exactly at compile time.
Moreover, it is possible to make a heterogeneous list -- where elements have different types -- and have operations with the tuple be type-stable.

In [ ]:
x = (1, "abc", 1.4)

In [ ]:
y = [x[1], x[2], x[3]]

In [ ]:
typeof(x) # element types are known exactly before checking

In [ ]:
typeof(y) # element types are only known once we index

#### NamedTuples

A `NamedTuple` is similar to a `Tuple` except each field has a specific name.
You can create a `NamedTuple` using key-value pairs within `(...):`

In [ ]:
z = (a=1.0, b="two", c=3)

Access elements using the dot syntax

In [ ]:
z.b

One particular use case for `NamedTuple` is for packaging several related variables into a single object, without having to write your own type.

#### Dict (Dictionaries)

A [`Dict`](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) in Julia is an object that maps a specific *key* to a unique *value* (key-value pairs).
Virtually anything can be used as a key or value, but you should prefer using immutable objects as keys to avoid headaches!

One way to define a `Dict` is by specifying key-value `Pairs`, `<key> => <value>`. Julia will infer the key and value types automatically.

In [ ]:
d = Dict("Alice" => 25, "Bob" => 22)

You can recover the value associated with a particular key using the usual indexing notation

In [ ]:
d["Alice"]

And update its values with assignment

In [ ]:
d["Bob"] = 33; d

### Modules and Packages: Using Other People's Code

#### Standard Library

Julia ships with its own standard library.
The functions immediately available mostly live in the Base module.
Other useful packages include: 

- LinearAlgebra, for everything linear algebra
- DelimitedFiles, for working with formatted files (e.g. tab or comma separated)
- Statistics, many standard statistics functions
- Random, random number generation
- Printf, for printing formatted strings (C-style)
- Pkg, the Julia pacakge manager

#### Registered Packages

Julia users and developers are free to contribute to the Julia package ecosystem by registering them.
The **General** registry belongs to the whole community, but it is also possible to create your own private registry that contains your own packages (hosting is a separate issue).

Regardless, you can manage community-contributed packages using the Pkg module.

Some examples include:

* DifferentialEquations
* Distributions
* DataFrames
* BenchmarkTools

and many, many more.

Here are a few tips for package management:

- Commands with Pkg and `]` (pkg-mode) are equivalent.
- `Pkg.add("Example")` or `] add Example` will add the Example package to your *environment*.
- `Pkg.status()` or `] status` will tell you the list of installed packages for the current environment.
- `Pkg.remove("Example")` removes a package.
- `Pkg.activate("path/to/dir")` will activate the environment at `path/to/dir`. Julia searches the specified folder for a `Project.toml` file, or uses an empty one if it does not exist.
- Use `Pkg.activate(".")` to return to the default environment.

#### Environments

The list of explicitly installed packages is stored in a file called `Project.toml`.
The main environment is usually called `v1.x` where `x` is the minor Julia version and lives in `~/.julia/environments/v1.x`.
This is important because it helps you reproduce the environment used in developing a project!

#### How to Actually Use Packages

Packages live inside [*modules*](https://docs.julialang.org/en/v1/manual/modules/). They are defined using module code blocks.

In [ ]:
module MyExample
    println("This module is loading")
    helloworld() = println("Hello world! This is my contribution.") # define a function
    println("We're done")

    export helloworld # export the function by default so others can use it
end

We see the code executed. Let's try calling the function we made:

In [ ]:
helloworld()

This fails because we've not loaded the module, only defined it.
You can use `import` to load it:

In [ ]:
import Main.MyExample # MyExample is defined in the Main scope, so we need to address it

In [ ]:
helloworld()

This still fails? The reason is `import` tells us to load the module, but prevents exported functions from being brought into scope. We have to be explicit about where the function comes from in this case.

In [ ]:
Main.MyExample.helloworld()

This can be a bit tedious, so fortunately Julia lets us bind a module to a variable to make it easier to write:

In [ ]:
ME = Main.MyExample

In [ ]:
ME.helloworld()

Alternatively, the `using` keyword brings all exported functions into scope:

In [ ]:
using Main.MyExample

In [ ]:
helloworld()

Yet another option: You can explicitly choose which functions to bring into scope when you ivoke `using`:

In [ ]:
using LinearAlgebra: norm # only bring norm into scope, unqualified

In [ ]:
norm(rand(3))

In [ ]:
eigen # lives in LinearAlgebra

In [ ]:
using LinearAlgebra

In [ ]:
eigen

# Projects

### Newtons' Method

Newton's method, also known as the Newton-Raphsons method, is an *iterative* technique that computes *roots* of a given function by exploiting information about its shape.
A root of $f$, or a zero of $f$, is simply a value $x$ such that $f(x) = 0$.
Given a function $f$ with derivative $f^\prime$, the algorithm can be summarized succintly as
$$
\underset{\text{next value in sequence}}{x_{k+1}}
=
\underset{\text{current value in sequence}}{x_{k}}
-
\underset{\text{Newton direction}}{\frac{ f(x_{k}) }{ f^\prime(x_{k}) }}.
$$
Starting from some initial guess for the root, $x_{0}$, one applies the rule above several times until the procedure converges to some specified tolerance level.

In [ ]:
function newton(f, df, x0; niters::Int=100, tol::Real=1e-2)
    # initialize
    f_init  = f(x0)                      # initial value for function values, f(x_0)
    df_init = df(x0)                     # initial value for derivative values, f'(x_0)
    f_prev  = typemax(f_init)            # previous function value, f(x_k)
    f_cur   = f_init                     # current function value, f(x_k+1)
    df_prev = typemax(f_init)            # previous function value, f'(x_k)
    df_cur  = df_init                    # current function value, f'(x_k+1)
    T = typeof(one(f_cur) / one(f_prev)) # infer common type
    x = convert(T, x0)                   # current iterate
    converged  = false                   # indicates whether we have achieved required tolerance
    iterations = 0                       # track the number of iterations taken

    for iter in 1:niters
        # iterate

        # update state

        # check convergence
    end

    return (x, iterations, converged)
end

In [ ]:
g(x) = x^2
dg(x) = 2*x
newton(g, dg, 4.0, tol=1e-16)

### Handling Derivatives

But what if you don't know all the calculus rules, or your function simply has a derivative that is hard to write down?

Fortunately, there are a number of techniques to
- compute derivatives exactly (automatic differentation),
- derive symbolic expressions (symbolic differentiation), and
- approximate derivatives numerically (finite differences).

In [ ]:
f(x) = tan(x)^2

#### Exact derivatives with ForwardDiff.jl

In [ ]:
using ForwardDiff

In [ ]:
fderiv1(x) = ForwardDiff.derivative(f, x)

In [ ]:
f(2)

In [ ]:
fderiv1(2)

In [ ]:
xs = range(-1, 1, step=1e-2)
plot(xs, [f, fderiv1],
    label=["f(x)" "f'(x)"],
    legend=:outerright,
    linewidth=3,
)

#### Approximate derivatives with FiniteDifferences.jl

In [ ]:
using FiniteDifferences

In [ ]:
fderiv2(x) = FiniteDifferences.central_fdm(2, 1)(f, x)

In [ ]:
f(2)

In [ ]:
fderiv2(2)

In [ ]:
xs = range(-1, 1, step=1e-2)
plot(xs, [f, fderiv2],
    label=["f(x)" "f'(x)"],
    legend=:outerright,
    linewidth=3,
)

#### Automatic Differenations vs Finite Differences

In [ ]:
abs(fderiv1(2) - fderiv2(2))

### Exercise: Write a function that compares forward mode AD with central finite differences.

#### Inputs:
- A function, call it `f`.
- A list of values, `xvalues`, at which to compare the two methods.

#### Outputs:

A single `DataFrame` with the following columns:

- `x`, which denotes the value at which to evalute the requested quantities.
- `f`, which represents the function value at a given point `x`.
- `FAD`, the value of the derivative based on ForwardDiff.jl.
- `CFD`, the value of the derivative based on the central differences method in FiniteDifferences.jl.
- `diff`, the absolute difference between the two methods.

**Bonus**

- Sort the data in ascending order based on the `x` column.
- Avoid the `push!` operation in the given `for` loop.
- Avoid writing an explicit `for` loop.

In [ ]:
function compare_derivatives(f, xvalues)
    # 1. Create objects that represent derivatives in ForwardDiff.jl and FiniteDifferences.jl.
    fp_FAD = x -> ForwardDiff.derivative(f, x)
    fp_CFD = x -> FiniteDifferences.central_fdm(2, 1)(f, x)
    
    # 2. Create objects to store different quantities.
    val_func = 
    val_FAD  =  # Your code goes here!
    val_CFD  =  # Hint: What data types are useful for representing lists or sequences?
    val_diff = 
    
    # 3. Compute the requested quantities for the summary.
    for x in xvalues
        push!(val_func, f(x))
        # Your code goes here!
    end
        
    # 4. Put everything into a DataFrame.
    # Your code goes here.
    
    # 5. Return the DataFrame object.
    return df # Your code goes here.
end

In [ ]:
test_func1(x) = x^2
xvalues = randn(5)

compare_derivatives(test_func1, xvalues)

In [ ]:
test_func2(x) = sin(x) / x
xvalues = [k*π/8 for k in 1:7]

compare_derivatives(test_func2, xvalues)

### Adding Visual Feedback

Now let's add a way to track progress in Newton's method.
The easiest way to do this would be to print a few values using `print` or `println`.
Here we will try to make our program modular by adding a step in the program that delegates the logging functionality to some other tiny program.
We will do this using what is known as a *function callback* -- a function passed to our program that will be called at some point using data from the Newton program.

In our code, we will add an argument, `cb`, which will be a *variable bound to a particular function*.
The underlying function will accept three arguments; that is, calling it will look like:

In [ ]:
cb(f, df, info)

where `f` is the function whose root we are interested in, `df` is the derivative of `f`, and `info` is an object that contains extra information.
In this case, we will use a `NamedTuple` to store information about the state of the algorithm (see the code below).

In [ ]:
function newton(f, df, x0; niters::Int=100, tol::Real=1e-2, cb::Function=DEFAULT_CB) # note the extra argument
    # initialize
    f_init  = f(x0)                      # initial value for function values, f(x_0)
    df_init = df(x0)                     # initial value for derivative values, f'(x_0)
    f_prev  = typemax(f_init)            # previous function value, f(x_k)
    f_cur   = f_init                     # current function value, f(x_k+1)
    df_prev = typemax(f_init)            # previous function value, f'(x_k)
    df_cur  = df_init                    # current function value, f'(x_k+1)
    T = typeof(one(f_cur) / one(f_prev)) # infer common type
    x = convert(T, x0)                   # current iterate
    converged  = false                   # indicates whether we have achieved required tolerance
    iterations = 0                       # track the number of iterations taken

    for iter in 1:niters
        # iterate
        x_prev = x
        x = x - f_cur / df_cur

        # update state
        f_prev  = f_cur
        df_prev = df_cur
        f_cur   = f(x)
        df_cur  = df(x)

        # package information into a NamedTuple?
        info = (
                iter=iter,
                x_cur=x,
                x_prev=x_prev,
                f_cur=f_cur,
                f_prev=f_prev,
                df_cur=df_cur,
                df_prev=df_prev,
        )
        
        # execute the callback
        cb(f, df, info)

        # check convergence
        converged = abs(f_cur - f_prev) < tol
        iterations += 1

        converged && break
    end

    return (x, iterations, converged)
end

# implement a default behavior for callback
DEFAULT_CB(f, df, info) = nothing

Check that everything still works:

In [ ]:
newton(g, dg, 4.0, tol=1e-8)

Now let's write a program that displays

1. the current iteration number,
2. the current estimate for the root, `x`, and
3. the current function value, `f(x)`.

We will use the `Printf` module to help us format our strings.
To understand the formatting, note that:

- `\t`: denotes a *tab character*.
- `\n`: denotes a *newline character*.
- `%3d`: denotes a number with 3 digits.
- `.8f`: denotes a floating point number has 8 digits after the decimal.

In [ ]:
using Printf

In [ ]:
function print_cb(f, df, info)
    x, fx, iter = info.x_cur, info.f_cur, info.iter

    # add a header for the first iteration
    iter == 1 && println("iter\tx\t\tf(x)")

    # print iteration, x value, and f(x) value
    @printf "%3d\t%.8f\t%.8f\n" iter x fx

    return nothing
end

In [ ]:
newton(g, dg, 4.0, tol=1e-8, cb=print_cb)

Our printing callback works as expected, but doing any analysis with the information printed out is cumbersome: you have to copy the information from your screen.

Next, we will implement a callback that actually stores the information for later use within Julia.
In order to avoid modifying our Newton's method program any further, we will use a slightly more advanced concept called a *closure*.
Specifically, we want to implement a function that stores information inside two arrays, but we need those arrays to be scoped within the calling function.

In [ ]:
# define the type, which is defined as a subtype of function so that it behaves like a function
struct MyLoggingCB{T} <: Function
    xvals::Vector{T} # the first field will store the x values
    fvals::Vector{T} # the second field will store the f(x) values
end

# provide a constructor to initialize both fields with empty arrays.
MyLoggingCB{T}() where T = MyLoggingCB(T[], T[])

# implement the desired behavior
function (cb::MyLoggingCB)(f, df, info)
    # flush if this is the first call
    if info.iter == 1
        empty!(cb.xvals)
        empty!(cb.fvals)
    end

    # log the data

    return nothing
end

In [ ]:
mycb = MyLoggingCB{Float64}()
newton(g, dg, 4.0, tol=1e-8, cb=mycb)

In [ ]:
xmin, xmax = extrema(mycb.xvals)
xs = range(xmin, xmax, length=1000)

plot(xs, g, lw=3, label="g(x)", legend=:outerright)
plot!(mycb.xvals, mycb.fvals, xlab="x", ylab="g(x)", label="path", arrow=true, lw=3, linestyle=:dash)

In [ ]:
# using Pkg; Pkg.add("Plots")
using Plots

struct MyPlottingCB{T} <: Function
    log::MyLoggingCB{T}
end

MyPlottingCB{T}() where T = MyPlottingCB{T}(MyLoggingCB{T}())

function (cb::MyPlottingCB)(f, df, info)
    # first log the new data
    cb.log(f, df, info)

    # now show a plot
    fig = plot(1:length(cb.log.fvals), cb.log.fvals, xlab="iteration", ylab="f(x)", lw=3)
    scatter!(1:length(cb.log.fvals), cb.log.fvals, legend=false, lw=0)
    
    # sleep a little so we can see that it happens as the algorithm progresses
    sleep(0.5)

    # make sure the figure is displayed
    display(fig)

    return nothing
end

In [ ]:
mycb = MyPlottingCB{Float64}()
newton(g, dg, 4.0, tol=1e-8, cb=mycb)

# Additional Topics

### Broadcasting

The `.` symbol has a special meaning in Julia in that it is reserved for two operations: accessing fields of a type, or *broadcasting*.
Roughly speaking, *broadcasting* allows you to apply functions to elements of a collection individually using a concise syntax.

For example we can do elementwise-multiplication:

In [ ]:
x = [1, 2, 3]
y = [12, 6, 4]

x .* y

This works for *any* function, not just operators:

In [ ]:
function foo(x)
    return x * sqrt(2)
end

In [ ]:
foo.(x)

Julia will try to reason about functions supporting multiple arguments where possible:

In [ ]:
function bar(x, y)
    return x + sqrt(y)
end

In [ ]:
bar.(x, y)

In [ ]:
z = 4 * ones(5)

In [ ]:
bar.(x, z)

Adding `.` everywhere can become tedious in a complicated expression, so Julia provides a special `@.` *macro* to automatically add broadasting to an expression:

In [ ]:
z = rand(3)

In [ ]:
@. x * y + z

In [ ]:
w = zeros(3)

for i in eachindex(w) # eachindex(w) is preferred style over 1:length(w)
    w[i] = x[i] * y[i] + z[i] # equivalent version
end

w

### Macros to know about

*Macros* in Julia are functions that transform Julia code into other Julia code.
This is possible in part because Julia is written in Julia itself, using `Symbol` and `Expr` objects.
For example:

In [ ]:
ex = :(1 + 3) # the : symbol can be used to define a symbol or expression

In [ ]:
typeof(ex)

In [ ]:
ex.head

In [ ]:
ex.args

In [ ]:
eval(ex) # evaluate the expression; you almost never have to use something like this in your code

The `@time` macro is useful for timing parts of your code.
To undrestand what we mean by code transformation, the `@time` macro is roughly equivalent to

In [ ]:
t1 = time() # get time in seconds
sleep(0.1) # do something
t2 = time() # get new time in seconds
(t2 - t1) # get time elapsed

In [ ]:
@time sleep(0.1); # includes information about memory allocation

Macros are quite an advanced topic, but here we will simply mention a few macros and describe their use:

1. `@time`: time a line of code (can be used on functions); see also [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) for more rigorous benchmarking.
2. `@show`: force displaying of a variable or expression
3. `@inbounds`: turn off bounds-checking on arrays within an expression (use with extreme care!)
4. `@code_warntype`: useful for checking Julia code for type stability
5. `@.`: Add `.` to functions in an expression to induce broadcasting
6. `@assert`: Check that an expression evaluates to true; otherwise throw an error.

### A little more on types

Julia has a flexible [type system](https://docs.julialang.org/en/v1/manual/types/).
As we've seen, a "type" roughly translates to "class" in object oriented programming (OOP).
In contrast to OOP, however, Julia types have *very* strict notion of inheritance: you can't extend concrete types.
Here we won't discuss how polymorphism or design work in Julia, but rather take a closer look at the anatomy of types.

There are three main classes of types to be aware of as you start with the language:

1. Composite Types: Types that combine other data types.
2. Parametric Types: Types that define a *family* of types, parameterized by some variables.
3. Abstract Types: These are just "nodes" in a hierarchy. You can define functions with them or use them in type annotations, but you cannot make instances.
For example, an `Int` is a subtype of `Real` so it sits somewhere in a hierarchy including both types.

Importantly, types can be defined at any point in time, but their structure cannot be changed once defined.
This means:

- You can edit functions that use a particular type, but not change what fields are part of the type after it's been defined.
- You can update a type's constructors. For our purposes, a constructor is merely a function that creates an instance of a type.

In the following code, note that `<name>::<type>` is a *type annotation* which declares that a particular variable has a certain type.
Type annotations can be used anywhere, including

- function signatures, which limit the type of arguments that can go into the function, and
- code blocks/function bodies, which enforce that an object must have a certain type.

In [ ]:
struct Point2D # define an (immutable) type named Point2D
    x::Float64 # has a field x which is a Float64
    y::Float64 # has a field y which is a Float64
end

In [ ]:
Point2D(1.0, 0.0) # Julia automatically defines a constructor for you

In [ ]:
struct ParPoint2D{T <: Real} # define a type family where type T must be a subtype Real number
    x::T # has a field x, which has type T
    y::T # has a field y, which is ALSO type T
end

In [ ]:
ParPoint2D(0,0) # type parameter automatically inferred

In [ ]:
ParPoint2D{Float64}(0, 0) # note the automatic conversion

We can define a no-argument constructor that defaults to the origin.
Note that here we must specify the type parameter using `{T}` and state that `T` is a free parameter using `where`.
Writing `where T` roughly translates to `where T <: Any` but we could also restrict `where T <: Real` if we want.

In [ ]:
ParPoint2D{T}() where T = ParPoint2D{T}(zero(T), zero(T)).

Note that this generic version works for many data types:

In [ ]:
@show ParPoint2D{Float64}()
@show ParPoint2D{Int}()

The alternative would be to write a no-argument constructor for every possible type value for `T`; e.g. `ParPoint{Float64} = ParPoint(0.0, 0.0)` which is tedious.
The generic version is guaranteed to work for any subtype of `Real`, *even types that did not exist when we implemented our type*.

One of the benefits of the Julia language is the ability to overload standard functions to take advantage of composition.
For example, it's possible to define a notion of addition with points by overloading `+`:

In [ ]:
import Base: + # existing functions must imported explicitly in order to overload them 

function Base.:+(A::ParPoint2D, B::ParPoint2D) # we could specify type parameters but there's no need
    return ParPoint2D(A.x + B.x, A.y + B.y)
end

ParPoint2D(1,1) + ParPoint2D(2,1)

Now anything that relies on `+` "just works":

In [ ]:
list = [ParPoint2D(1,1), ParPoint2D(2,1), ParPoint2D(3,0)]
sum(list)

### (Im)mutability and function name conventions

Every type in Julia is either *mutable* or *immutable*.
A type is said to be *mutable* if you are allowed to change its contents.
In contrast, *immutable* objects cannot have their contents changed.

For example, if we create a variable that stores the number `1` we can bind the variable to a new value *but its contents do not change* because it is immutable.
Let's illustrate this by writing a wrapper type for `Int`:

In [ ]:
struct MyInt # structs are immutable
    value::Int
end

In [ ]:
x = MyInt(1)

In [ ]:
x.value = 2

In [ ]:
mutable struct MutableInt
    value::Int
end

In [ ]:
y = MutableInt(1)

In [ ]:
y.value = 2

In [ ]:
y

Given this behavior, you may naturally wonder how objects may or may not change if one attempts to mutate them inside a function.

In [ ]:
a = [1.0, 2, 3]

In [ ]:
function mutate_second_element(x)
    x[2] = sqrt(2) # this is MUTATION! it is shorthand for setfield!(x, new_value, index)
end

In [ ]:
mutate_second_element(a)

In [ ]:
a

In [ ]:
@which a[2] = sqrt(2)

In [ ]:
function assign_and_mutate_second_element(x)
    y = x # this is ASSIGNMENT
    y[2] = sqrt(3) # this is MUTATION
end

In [ ]:
assign_and_mutate_second_element(a)

In [ ]:
a

To understand this example, remember that `=` by itself simply *binds* a value to a particular symbol.
So, `y = x` simply means that the data attached to `x` is now also attached to `y`; no new copy is created.
Mutating the data attached to `y` is equivalent to mutating `x`.

We can test this with another example:

In [ ]:
function copy_and_mutate_second_element(x)
    y = copy(x) # assign y a copy of the data in x
    y[2] = sqrt(17)
end

In [ ]:
copy_and_mutate_second_element(a)

In [ ]:
a

**To help programmers understand that objects will be mutated after calling a particular function, we use the convention of adding `!` to the end of a mutating function**. This lets users know that a function will mutate some arguments. Usually, we put the mutated arguments on the left of the function signature.

In fact, it is a common pattern in Julia to separate a function into allocating and non-allocating versions:

In [ ]:
function compute_partial_sums(x)
    # create an Array with the same shape as x, but filled with zero(eltype(x)). 
    partial = zero(x)
    
    # do the actual work, storing the result in partial
    # we return whatever this function evaluates to
    compute_partial_sums!(partial, x)
end

In [ ]:
function compute_partial_sums!(partial, x) # use convention; has ! and mutated argument is the first
    for i in eachindex(x)
        prev = i > 1 ? partial[i-1] : zero(eltype(x))
        partial[i] = prev + x[i]
    end
    return partial
end

In [ ]:
x = [1, 1, 1, 1]

In [ ]:
compute_partial_sums(x)

This is essentially what the `cumsum` (cumulative sum) function does:

In [ ]:
cumsum(x)

In [ ]:
partial = zero(x)
cumsum!(partial, x)
partial

### Type stability

A block of code, such as a function, is said to be *type-stable* if Julia is able to determine the types of all variables at compile time.
Having complete type information allows Julia's compiler to automatically generate efficient code that runs on your machine.
You can investigate type stability using the `@code_warntype` macro, which will highlight potential type instability in red (serious) and yellow (okay).

In [ ]:
function silly_example(x)
    if x < 10
        return "ten"
    else
        return x
    end
end

In [ ]:
silly_example(1)

In [ ]:
silly_example(12.3)

In [ ]:
@code_warntype silly_example(1)

Here the annotation with `Union{Int64,String}` means that the return value is either a `Int64` or a `String`, which is called a *type union*.
Julia handles minor type stability violations by introducing type unions to limit the possible space of return types.
In the *worst case* the return type will be `Any`, the catch-all when Julia is unable to infer any type information.

Let's look at a more realistic example.
Here we have a function that computes the sum of an array:

In [ ]:
function mysum(x)
    value = 0 # initialize
    for i in eachindex(x) # iterate over the indices in x
        value = value + x[i] # sum
    end
    return value
end

In [ ]:
x = [1.0, 2.0, 3.0]
mysum(x)

In [ ]:
@code_warntype mysum(x)

This means the return type for `value` is either a `Int64` or a `Float64`.
The reason is that `value = 0` is initially an `Int64`. Then:
- if the `for` loop runs, `value = value + x[1]` where `x[1]::Float64` so `value` is promoted to a `Float64`, or
- if the `for` loop never runs, then `value = 0` which is an `Int64`.
This seems like a bad consequence of Julia's flexible type system.
However, it has a very easy fix:

In [ ]:
function mysum(x)
    value = zero(eltype(x)) # set value to the zero of the same type as elements in x
    for i in eachindex(x)
        value = value + x[i]
    end
    return value
end

In [ ]:
x = Int64[1, 2, 3]
@code_warntype mysum(x)

In [ ]:
x = Float32[1, 2, 3]
@code_warntype mysum(x)

The type instability is gone!
Now we have a generic, type-stable function that preserves types after the sum reduction.

### Functions vs Methods

A *function* in Julia is essentially the symbol bound to a particular "action".
A *method* is a particular version or implementation of that action.

For example, the `sqrt` function will behave differently depending on the input type.

In [ ]:
?sqrt

Julia lets us look up *methods* for a particular *function* using `methods`:

In [ ]:
methods(sqrt)

This table shows us that the `sqrt` function has 19 different methods, each specializing on different input types.

In [ ]:
sqrt(-1.0)

In [ ]:
sqrt(Complex(-1.0))

### Optional arguments in functions

It is possible to define functions with optional arguments.
There are two kinds to consider:

* *Positional arguments* occur at the end of a function signature in a specific order and have default values. If the optional arguments are not specified, the default values are used.
* *Keyword arguments* begin after a semicolon `;` in a function signature. Changing the default values requires the caller to name the arguments and assign non-default values.

You can specify as many optional arguments as needed, and can mix positional and keyword arguments.

In [ ]:
function add_number(a, b=1)
    return a + b
end

In [ ]:
function also_add_number(a; b=1)
    return a + b
end

In [ ]:
add_number(2) # don't specify the positional argument b

In [ ]:
add_number(2, b=2) # this fails

In [ ]:
add_number(2, 2) # specify b

In [ ]:
also_add_number(2) # don't specify the keyword argument b

In [ ]:
also_add_number(2, 2) # this fails

In [ ]:
also_add_number(2, b=2) # the keyword argument must be invoked explicitly

### Duck Typing and Multiple Dispatch

In all our functions so far we have not specified the *types* of arguments that go into the function.
Yet, Julia is always aware of the types of arguments and variables within a function in spite of this fact.
This is known as *duck typing* and is actually a nice feature common to dynamic languages because it allows functions to be written in a generic fashion.

In many other programming languages, particularly those supporting object oriented design as it is commonly understood, there is the notion of *dispatching* to the most specific method in order to fulfill a function call.
The `sqrt` example with `Float64` and `Complex{Float64}` are examples of this.

However, this example involves pattern matching on just one argument. What's the "multiple" in "multiple dispatch" and how is it related to duck typing in Julia?

To understand this, consider the following example from [Ole Kröger](https://opensourc.es/blog/basics-multiple-dispatch/) ([adapted from an example by Julia co-creator Stefan Karpinski](https://www.youtube.com/watch?v=kc9HwsxE1OY)):

In [ ]:
abstract type Animal end

# define a Lizard type with a name field
struct Lizard <: Animal
    name :: String
end

# define a Rabbit type with a name field
struct Rabbit <: Animal
    name :: String
end

# Define the race function with 3 methods: 2 specific ones and one generic fallback
race(l::Lizard, r::Rabbit) = "$(l.name) wins in wall climbing."
race(r::Rabbit, l::Lizard) = "$(r.name) wins in a normal race."
race(a::T, b::T) where T <: Animal = "$(a.name) and $(b.name) run forever."

# Define a meet function that calls the race function.
function meet(a::Animal, b::Animal) 
    println("$(a.name) meets $(b.name) and they race!")
    println("Outcome: $(race(a,b))")
end

# Try it out with our two Animal types
bayi = Lizard("Bayi")
sally = Rabbit("Sally")
meet(bayi, sally)  # calls the first race method
meet(sally, bayi)  # calls the second race method
meet(sally, sally) # calls the generic fallback

Multiple dispatch lets Julia and its compiler *select the most specific method **based on the runtime types**.*
To someone coming from C++, this example makes multiple dispatch look just like function overloading.

However, implementing this code in C++ and running it reveals that C++ will always call the third, generic fallback where two animals run forever! This is because while C++ can choose a specific method, it is limited to the type information **available at compile time**. As such, adding the third method effectively erases methods one and two.

The workaround in C++ would be to make the `race` function specific to the abstract/superclass `Animal` class; i.e. we need to be able to write `bayi.race(sally)` and `sally.race(bayi)`.
This seems reasonable, but you will run into trouble as soon as you need to make an `Animal` race a `Human` because now both classes must have `race` as part of their definitions, but you may not have designed the classes with this use case in mind!

*In object-oriented languages, methods belong to class functions not just functions in general.*

In contrast, in Julia this would be simply a matter of adding `race(l::Human, r::Animal)` to handle cases as needed.
Thus, Julia rewards you for writing "weakly" or duck typed functions by letting you handle specific cases with different input types only as needed, without having to completely redesign your code.
**In fact, the preferred style in Julia is to avoid adding type annotations in your code unless absolutely necessary**.

These two points make code reuse much more probable, and is the "magic" that lets packages build lots of nice, unexpected features into them. (Feel free to ask for examples!)